In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext  tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score  
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd 'drive/My Drive/Colab Notebooks/matrix/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_road_sign


In [5]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [6]:
 if (y_train.ndim == 1):
     y_train = to_categorical(y_train)

 if (y_test.ndim == 1):
     y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [7]:
def train_model(model, X_train, y_train, params_fit = {}):
    model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)
    model.fit(
        X_train, 
        y_train,
        batch_size = params_fit.get('batch_size', 128),
        epochs = params_fit.get('epochs', 5),
        verbose = params_fit.get('verbose', 1),
        validation_data = params_fit.get('validation_data', (X_train, y_train)),
        callbacks = [ tensorboard_callback ]
    )

    return model

In [8]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
    y_test_norm = np.argmax(y_test, axis = 1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax( y_pred_prob, axis = 1)

    return scoring(y_test_norm, y_pred)


In [9]:
def get_ccn_v5(input_shape, num_classes):
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape ),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),
        
        Dense(1024, activation = 'relu'),
        Dropout(0.3),
        
        Dense(1024, activation = 'relu'),
        Dropout(0.3),

        Dense(num_classes, activation = 'softmax'),
    ])


In [10]:
model = get_ccn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 48ms/step - loss: 2.6638 - accuracy: 0.2756 - val_loss: 1.2299 - val_accuracy: 0.5986
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 1.0495 - accuracy: 0.6534 - val_loss: 0.3282 - val_accuracy: 0.9084
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4949 - accuracy: 0.8435 - val_loss: 0.1162 - val_accuracy: 0.9643
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2968 - accuracy: 0.9088 - val_loss: 0.0970 - val_accuracy: 0.9704
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2243 - accuracy: 0.9336 - val_loss: 0.0349 - val_accuracy: 0.9896


0.9709750566893424

In [12]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.1352 - accuracy: 0.9710


0.9709750413894653

In [46]:
def get_model(params):
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape ),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),
        
        Dense(1024, activation = 'relu'),
        Dropout(params['dropout_dense_block_one']),
        
        Dense(1024, activation = 'relu'),
        Dropout(params['dropout_dense_block_two']),

        Dense(num_classes, activation = 'softmax'),
    ])


In [44]:
def func_obj(params):
    model = get_model(params)

    model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

    model.fit(
        X_train, 
        y_train,
        batch_size = int( params.get('batch_size', 128)),
        epochs = 5,
        verbose = 0,
        
    )
    score = model.evaluate(X_test, y_test, verbose = 0)
    accuracy = score[1]
    print(params, 'accuracy={}'.format(accuracy))

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model }

In [47]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best= fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials(),

)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.49853623206142783, 'dropout_cnn_block_three': 0.38991186304633396, 'dropout_cnn_block_two': 0.48552874913590727, 'dropout_dense_block_one': 0.5423506174707452, 'dropout_dense_block_two': 0.357011379259474}
accuracy=0.9335600733757019
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.382814220991926, 'dropout_cnn_block_three': 0.3152577578283748, 'dropout_cnn_block_two': 0.3941530494193449, 'dropout_dense_block_one': 0.5350347157824041, 'dropout_dense_block_two': 0.41478395216773056}
accuracy=0.9498866200447083
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4874602443409918, 'dropout_cnn_block_three': 0.329204691005776, 'dropout_cnn_block_two': 0.48896405104929347, 'dropout_dense_block_one': 0.3614114700696036, 'dropout_dense_block_two': 0.6777394368877472}
accuracy=0.9290249347686768
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3124260637875095, 'dropout_cnn_block_three': 0.47029228971947507, 'dropout_cnn_block_two': 0.3499865987009

In [48]:
!git add dw_matrix_three/day5

data/  day1.ipynb  dw_matrix_three/  LICENSE  logs/  README.md
